In [ ]:
#카카오페이지

In [53]:
import bs4
import requests
import urllib.request
from selenium import webdriver
import bs4
import requests
from urllib import parse
import time
import re


print("작품이름 -> ")
search_name=input()
print("\n")

tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "https://page.kakao.com/search?word=" + parse.quote(tmp)


r=requests.get(url)
c=r.content
bs_obj = bs4.BeautifulSoup(c, "html.parser")

boxes = bs_obj.findAll("div", {"class": "css-c09e5i"})

for box in boxes:
    is_novel=box.find("div",{"class":"css-vurnku"}).text
    if("소설" in is_novel):
        parent=box.parent.parent
        title=parent.find("div", {"class": "text-ellipsis css-11602e0"}).text.replace(" ","")
        if("단행본" in title):
            continue
        elif(search_name_nonspace not in title):
            continue
        else:
            main=parent.parent.parent
            monopoly=True
    else:
        continue
title=main.find("div", {"class": "text-ellipsis css-11602e0"}).text

if '[' in title:
    print("작품이름 : " + title.split('[')[0])
    print("독점여부 : " + title.split('[')[1][:-1])
else:
    print("작품이름 : " + title)
    print("독점여부 : 미독점")

print("감상인원 : " + main.find('div',{'class','css-zlhhis'}).text)
print("장르 : " + main.findAll('div',{'class','css-vurnku'})[1].text)
print("작가 : " + main.find('div',{'class','text-ellipsis css-1pysja1'}).text)
#print("작가 : " + main.find('div',{'class','css-text-ellipsis css-1pysja1'}).text)

latest=main.find('div',{'class','text-ellipsis css-1eorkc4'})

if latest.text!="완결":
    print("완결여부 : 미완결")
    print("최근업데이트일 : " + latest.text.replace(" 업데이트",""))

else:
    print("완결여부 : " + latest.text)

image=main.find('div',{'class','jsx-922166243 imageWrapper'}).find('img')
print("이미지 : " + image.attrs['data-src'])


age=main.find("div", {"class": "css-fe9s02"}).find('img')
if age==None:
    print("연령등급 : 전체이용가")
else:
    print("연령등급 : 15세이용가")


pages=set()

for link in main.findAll("a", href = re.compile('^(/home)((?!:).)*$')):
    if 'href' in link.attrs: # 위에서 찾은 link에 href 속성이 있는지 확인   
        if link.attrs['href'] not in pages: # 새로운 페이지인지 확인                 
            newPage = link.attrs['href']
    
#기다무소설or 소설
#[단행본] X
#[닥터최태수 or 닥터 최태수 포함]

#작품의 링크
url="https://page.kakao.com" + newPage 
driver = webdriver.Chrome("C:/chromedriver.exe")
driver.get(url)
bs_obj = bs4.BeautifulSoup(driver.page_source, "html.parser")

driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[2]/button[1]').click()

time.sleep(0.2)

if latest.text!="완결":
    update_days = driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[1]/div[2]/div[2]/div[1]/p[1]')
    print("연재 요일 : " + update_days.text.replace(" 연재",""))
    
else:
    print("연재 요일 : 연재 종료")
time.sleep(0.2)

publisher=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[3]/div[2]')
#  두사랭                             '/html/body/div[2]/div/div/div/div[2]/div/div[2]/table/tbody/tr[1]/td[2]/div[3]/div[2]'


#두사랭 selector
# -> body > div:nth-child(32) > div > div > div > div.jsx-4041896147.modalBody > div > div.jsx-3248037293.infoTableBox > table > tbody > tr.jsx-3248037293.first > td.jsx-3248037293.contentCol > div:nth-child(3) > div.jsx-3248037293.publisherName
#닥최 selector
# -> body > div:nth-child(32) > div > div > div > div.jsx-4041896147.modalBody > div > div > table > tbody > tr.jsx-3248037293.first > td.jsx-3248037293.contentCol > div:nth-child(3) > div.jsx-3248037293.publisherName

print("출판사 : " + publisher.text)

total=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/div[1]/span[1]')
print("연재 수 : "+ total.text.replace('전체(',"").replace(')',""))
time.sleep(0.2)

first=driver.find_element_by_xpath('//*[@id="root"]/div[3]/div/div/div[3]/ul/li[1]/div[2]/dl/dd')
print("최초 업데이트일 : " + first.text)
time.sleep(0.2)


intro=driver.find_element_by_xpath('/html/body/div[2]/div/div/div/div[2]/div/div/table/tbody/tr[1]/td[2]/div[1]')
print("작품소개\n" + "----------------------------------------\n" + intro.text + "\n----------------------------------------")
time.sleep(0.2)

#driver2.quit()

작품이름 -> 
닥터최태수


작품이름 : 닥터 최태수
독점여부 : 미독점
감상인원 : 190.8만명
장르 : 현판
작가 : 조석호
완결여부 : 미완결
최근업데이트일 : 2020.04.20
이미지 : //dn-img-page.kakao.com/download/resource?kid=bhMa3y/hyzTXzI4xo/ruOHqz5PVlrWZ9IWcN7sBk&filename=th1
연령등급 : 전체이용가
연재 요일 : 연재 중
출판사 : 시나브로
연재 수 : 3667
최초 업데이트일 : 2015.06.06
작품소개
----------------------------------------
“그래. 환자를……. 무서워해야 돼. 두려워하고 손발이……. 떨리는 공포도 느껴야지.”
“…….”
“거기서 도망치지……. 않아야 진짜 의사가 되는 거야.”
“말씀하시면 출혈이 계속돼요.”
다급함에 젖어든 반항적인 태수의 말투에도 노인은 미소를 잃지 않고 말했다.
“언제고 어느 때고……. 도망치고 싶을 때가 올 거야. 그땐 지금을……. 떠올려. 환자가 의지할 사람은……. 자네 한 사람 뿐이라는 걸 말이야.”
삶과 죽음이 교차하는 그 시간이 지났다.
그후!!!!!!
평범했던 한 인턴의 운명이 송두리째 바뀌었다.

살릴 수 있다면?
살릴 가능성이 있다면?
어떤 상황에서라도 메스를 들어야 한다.
그런 불굴의 신념 하나면 충분하다.
닥터 최태수, 그는 진정한 외과의사의 길을 택했다.
----------------------------------------


In [ ]:
#문피아

In [5]:
import bs4
import requests
from urllib import parse

print("작품이름 -> ")
search_name=input()
tmp = search_name
search_name_nonspace=search_name.replace(" ","")

url = "https://novel.munpia.com/page/hd.platinum/view/search/keyword/" + parse.quote(tmp) + "/order/search_result"


r=requests.get(url)
c=r.content
html = bs4.BeautifulSoup(c, "html.parser")

article=html.find('div',{'class','article'})
dl=article.find('dl',{'class','detail'})
href=dl.find('a').attrs['href']


url = href

r=requests.get(url)
c=r.content
html = bs4.BeautifulSoup(c, "html.parser")

print("\n")

intro=html.find('p',{'class','story'})
print("작품소개\n" + "----------------------------------------\n" + intro.text + "\n----------------------------------------")

cover_box=html.find('div',{'class','cover-box'})
image=cover_box.find('img').attrs['src']
print("이미지 : " + image)

box=html.find('div',{'class','dd detail-box'})
h2=box.find('h2')
monopoly=h2.find('span')
if(monopoly==None):
    print("독점여부 : 미독점")
    title=h2.text.replace('\n',"")
    print("제목 : " + title)
    #print("제목 : " + title,end='')
else:
    print("독점여부 : " + monopoly.text)
    title=h2.text.split("독점\n")[1]
    print("제목 : " + title,end='')
    
p=box.find('p',{'class','meta-path'})
genre=p.find('strong').text
print(genre)
if ',' in genre:
    print("장르 : " + genre.split(',')[0] + ", " + genre.split(',')[1] )
else:
    print("장르 : " + genre)

fr=box.find('div',{'class','fr'})
prefer=fr.find('b').text
print("선호수 : " + prefer)

days = dict.fromkeys(["mon", "tue", "wed", "thu", "fri", "sat", "sun"], False)
novel_period=box.find('div',{'class','novel-period'})
period=""
if novel_period!=None:
    if(novel_period.find('span',{'class','pluszone-mon'})):
        days.update(dict.fromkeys(["mon"], True))
        period=period+"월,"
    if(novel_period.find('span',{'class','pluszone-tue'})):
        days.update(dict.fromkeys(["tue"], True))
        period=period+"화,"
    if(novel_period.find('span',{'class','pluszone-wed'})):
        days.update(dict.fromkeys(["wed"], True))
        period=period+"수,"
    if(novel_period.find('span',{'class','pluszone-thu'})):
        days.update(dict.fromkeys(["thu"], True))
        period=period+"목,"
    if(novel_period.find('span',{'class','pluszone-fri'})):
        days.update(dict.fromkeys(["fri"], True))
        period=period+"금,"
    if(novel_period.find('span',{'class','pluszone-sat'})):
        days.update(dict.fromkeys(["sat"], True))
        period=period+"토,"
    if(novel_period.find('span',{'class','pluszone-sun'})):
        days.update(dict.fromkeys(["sun"], True))
        period=period+"일,"
    
    
    if(all(value==True for value in days.values())==True):
        period="매일 "
    
    print("연재 주기 : " + period[:-1] + " 연재")

   

else:
    print("연재 주기 : 비주기적 연재")

    
age=box.find({'class','xui-icon xui-adult'})
if(age==None):
    print("연령등급 : 전체이용가")
else:
    print("연령등급 : " + age.text)
    

iconset=box.find('p',{'class','iconset'})
finish=iconset.find('span',{'class','xui-icon xui-finish'})
if finish!=None:
    print("완결여부 : " + finish.text)
else:
    print("완결여부 : 미완결")

writer_dl=box.find('dl',{'class','meta-author meta'})
writer=writer_dl.find('strong').text
print("작가 : " + writer)

etc_dl=box.findAll('dl',{'class','meta-etc meta'})
days_dl=etc_dl[0]
days_dd=days_dl.findAll('dd')
first_update=days_dd[0].text
print("작품등록일 : " + first_update)
recently_update=days_dd[1].text
print("작품등록일 : " + recently_update)

number_dl=etc_dl[1]
number_dd=number_dl.findAll('dd')

total=number_dd[0].text
print("연재수 : " + total)
recommend=number_dd[1].text
print("추천수 : " + recommend)
enjoyer=number_dd[2].text
print("조회수 : " + enjoyer)

작품이름 -> 
1983


작품소개
----------------------------------------
만화광인 이윤환.
느닷없이 1983년, 자신과 같은 이름, 같은 모습의 인간으로 빙의되어 버렸다.

“오래된 만화를 좋아하긴 해도, 1980년대로 보내버리면 나더러 어쩌라는 거야?”

80년대에서 만화덕후의 삶이 시작된다.
----------------------------------------
이미지 : //cdn1.munpia.com/files/attach/2018/1227/004/NT1qYG4RErsKnjnv.jpg
독점여부 : 선독점
제목 : 1983 전생 만화왕
현대판타지
장르 : 현대판타지
선호수 : 13,585
연재 주기 : 비주기적 연재
연령등급 : 전체이용가
완결여부 : 완결
작가 : 장성필
작품등록일 : 2019.01.18 17:00
작품등록일 : 2020.03.06 08:05
연재수 : 425 회
추천수 : 5,303,689
조회수 : 193,316


In [ ]:
"""
정확한 제목이 아닐때 작품을 선택할 수 있는 기능 추가 필요할듯
ex) 1983 검색
1983 전생만화왕
1983 전생만화왕[E]
게임 마켓 1983
게임 마켓 1983[E]
"""


# 15세 이상이면 해당 사이트에 (팝업창)로그인을 할 수 있게끔, 그리고 초기페이지로부터 검색하는 시스템 같은게 필요할듯?
# https://dejavuqa.tistory.com/272
# https://m.blog.naver.com/CommentList.nhn?blogId=tjdrud1323&logNo=221791791954
# 로그인 정보는 이용되지 않으며, 작품 정보 검색시에만 활용됩니다.

# 완결했으면 최근업데이트 따로 XX



#19세는 로그인이 필요한데 해당 세션 하는법 찾아야함.
#https://devlog.jwgo.kr/2018/02/21/how-to-get-login-session-in-python/